In [1]:
import os
from glob import glob
from scipy.io import loadmat
import pandas as pd
import numpy as np
import json
from PIL import Image
import io

from tqdm import tqdm

import pandas as pd

import matplotlib.pyplot as plt
import tensorflow as tf

% matplotlib inline

C:\Users\junhoning\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
mat_path = 'mpii_human_pose_v1_u12_1.mat'
image_datas = glob('mpii_samples/*.jpg')
mat = loadmat(mat_path)

In [9]:
def get_head_rect(path):
    path_name = os.path.basename(path)

    file_n = 0
    found = False
    while found == False:
        anno = mat['RELEASE']['annolist'][0, 0][0][file_n]
    #     anno['annorect'].dtype
        filename = anno['image']['name'][0, 0][0]
        if filename == path_name:
            found = True

        file_n += 1

    if 'x1' in str(anno['annorect'].dtype):
        head_rect = (True, [
            [x1[0, 0] for x1 in anno['annorect']['x1'][0]],
            [y1[0, 0] for y1 in anno['annorect']['y1'][0]],
            [x2[0, 0] for x2 in anno['annorect']['x2'][0]],
            [y2[0, 0] for y2 in anno['annorect']['y2'][0]]])
    else:
        head_rect = (False, [[0], [0], [0], [0]])
        
    return head_rect

In [10]:
def _int64_feature(value):
    """Wrapper for inserting int64 features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    """Wrapper for inserting float features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [11]:
tfrecords_name = 'facenet.tfrecords'

# options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
writer = tf.python_io.TFRecordWriter(path=tfrecords_name)

In [12]:
df_concat = []
for n, path in tqdm(enumerate(image_datas)):    
    head_exit, head_rect = get_head_rect(path)
    
    head_rect = np.array(head_rect, dtype=np.int64).tolist()
    
    if head_exit:
        with tf.gfile.GFile(path, 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        
        
        data = Image.open(path)  # .convert('RGB')
        image = np.array(data)

        height = image.shape[0]
        width = image.shape[1]

        filename = os.path.basename(path)
        image_format = 'jpg'.encode()
        

        x_min = (np.array(head_rect[0]) / width).tolist()
        x_max = (np.array(head_rect[2]) / width).tolist()
        y_min = (np.array(head_rect[1]) / height).tolist()
        y_max = (np.array(head_rect[3]) / height).tolist()

        classes_text = [str.encode('face')] * len(x_min)
        classes = [1 for label in x_min]
        
        string_set = tf.train.Example(features=tf.train.Features(feature={
            'image/filename': _bytes_feature(filename.encode()),
            'image/height': _int64_feature(height),
            'image/width': _int64_feature(width),
            # tf.compat.as_bytes(image.tostring())
            'image/encoded': _bytes_feature(encoded_jpg),
            'image/source_id': _bytes_feature(image_format),

            'image/object/bbox/xmin': _float_feature(x_min),
            'image/object/bbox/xmax': _float_feature(x_max),
            'image/object/bbox/ymin': _float_feature(y_min),
            'image/object/bbox/ymax': _float_feature(y_max),
            'image/object/class/text': _bytes_feature(classes_text),
            'image/object/class/label': _int64_feature(classes),
        }))

        writer.write(string_set.SerializeToString())

writer.close()

8it [00:00, 20.05it/s]
